Initial Imports:

In [209]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.transforms import Normalize
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.cluster import SpectralClustering
from sklearn.metrics import silhouette_score
from sklearn.decomposition import NMF
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA

1.***[Loading the data]***:
For MNIST:
In Python using TensorFlow and Keras:

In [58]:
# Transformations to apply to the dataset
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.5,), (0.5,))  # Normalize the data
])

# Load the MNIST dataset
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

For CIFAR-10:


In [59]:
# Transformations for CIFAR-10
transform_cifar = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the data
])

# Load the CIFAR-10 dataset
cifar_trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_cifar)
cifar_testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_cifar)


Files already downloaded and verified
Files already downloaded and verified


Paper results:

In [60]:
file_path = 'paper_results.csv'

# Read the CSV file into a Pandas DataFrame
data = pd.read_csv(file_path)
print("Results obtained in the paper:")
data

Results obtained in the paper:


,Method,MNIST ACC,MNIST NMI,USPS ACC,USPS NMI,CIFAR-10 ACC,CIFAR-10 NMI
0,K-means,0.58,0.49,0.48,0.42,0.14,0.12
1,Deep Cluster,0.86,0.83,0.67,0.69,NA,NA
2,Deep K-means,0.84,0.80,0.76,0.78,NA,NA
3,CSC No Flatten,0.85,0.79,0.83,0.78,0.12,0.08
4,CSC No Filter,0.83,0.76,0.84,0.79,0.14,0.10
5,CSC No Voting,0.82,0.77,0.82,0.76,0.14,0.10
6,CSC,0.86,0.81,0.83,0.79,0.15,0.11


### A - 1) Normalization

In [61]:
# Data normalization with min-max method
def min_max_scale_dataset(dataset, batch_size=64):
    
    min_pixel_value = float('inf')
    max_pixel_value = float('-inf')

    for images, _ in DataLoader(dataset, batch_size=batch_size, shuffle=True):
        min_pixel_value = min(min_pixel_value, images.min())
        max_pixel_value = max(max_pixel_value, images.max())

    
    min_max_scaler = Normalize(min_pixel_value, max_pixel_value)
    # Apply the min-max scaling to the dataset
    transform = transforms.Compose([transforms.ToTensor(), min_max_scaler])
    normalized_dataset = MNIST(root='./data', train=True, download=True, transform=transform)

    normalized_loader = DataLoader(normalized_dataset, batch_size=batch_size, shuffle=True)

    return normalized_loader

In [62]:
BATCH_SIZE = 64

In [63]:
mnist_train_loader = min_max_scale_dataset(mnist_trainset)
mnist_test_loader = min_max_scale_dataset(mnist_testset)

### A - 2) Autoencoder

The autoencoder architecture

In [64]:
#Defining the model
class ConvAutoencoder(nn.Module):
    def __init__(self, input_channels):
        super(ConvAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(16, input_channels, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid(),
        )

        self.bottleneck = nn.Linear(16 * 14 * 14, 500)

    def forward(self, x):
        x = self.encoder(x)
        flatten_x = x.view(x.size(0), -1)  # Flatten the output from the encoder
        bottleneck_features = self.bottleneck(flatten_x)
        reconstructed = self.decoder(x)
        return bottleneck_features, reconstructed

In [65]:
input_channels_mnist = 1  # MNIST images are grayscale
input_channels_cifar = 3  # CIFAR-10 images have 3 channels (RGB)

mnist_autoencoder = ConvAutoencoder(input_channels_mnist)
cifar_autoencoder = ConvAutoencoder(input_channels_cifar)

In [66]:
criterion = nn.MSELoss()
optimizer_mnist = torch.optim.Adam(mnist_autoencoder.parameters(), lr=0.001)
optimizer_cifar = torch.optim.Adam(cifar_autoencoder.parameters(), lr=0.001)

The loop for training the model

In [67]:
NUM_EPOCH = 10
def training_loop (model, loader, optimizer):
    model.train()

    for epoch in range(NUM_EPOCH):
        
        for data in loader:
            img, _ = data
            optimizer.zero_grad()
            _, output = model(img)
            loss = criterion(output, img)
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch + 1}/{NUM_EPOCH}], Loss: {loss.item():.4f}')

    return model

In [68]:
mnist_autoencoder = training_loop(mnist_autoencoder,mnist_train_loader,optimizer_mnist)
#cifar_autoencoder = training_loop(cifar_autoencoder,cifar_train_loader,optimizer_cifar)

Epoch [1/10], Loss: 0.0026
Epoch [2/10], Loss: 0.0012
Epoch [3/10], Loss: 0.0008
Epoch [4/10], Loss: 0.0006
Epoch [5/10], Loss: 0.0006
Epoch [6/10], Loss: 0.0005
Epoch [7/10], Loss: 0.0004
Epoch [8/10], Loss: 0.0004
Epoch [9/10], Loss: 0.0004
Epoch [10/10], Loss: 0.0004


# The evaluation loop to extract 500 features from the bottleneck layer for each input image

In [69]:
def evaluation (model, loader):
    '''
    output shape : bottelneck_feature (nb img,500)
    '''
    model.eval()

    bottleneck_features_array = []
    outputs_array = []
    total_loss = 0
 
    for data in loader:
        img, _ = data
       
        bottleneck_features, output = model(img)

        for features in bottleneck_features:
            bottleneck_features_array.append(features.detach().numpy())
        outputs_array.append(output.detach().numpy())

        loss = criterion(output, img)
        total_loss += loss.item()

    average_loss = total_loss / len(loader)
    print("Evaluation loss : ",average_loss)
    
    return np.array(bottleneck_features_array), outputs_array


### B - Best features selection

Do the Non negativ Matrix Factorization (NMF) to estimate a decomposition W*H from V \
Calcul the error rate E from W*H+E = V \
Sort feature by error rate and keep the 50% best

In [70]:
def feature_filtering_nmf(features, k=1, removal_percentage=50):
    '''
    output shape : (nb img,250)
    '''

    #it doesn't work because of negative value so we remove the negative here
    min_value = np.min(features)
    features_shifted = features - min_value + 1e-10 

    # NMF : find the decomposition V=W*H
    nmf_model = NMF(n_components=k, init='random', random_state=None)
    nmf_features = nmf_model.fit_transform(features_shifted)

    #reconstruc W*H to evaluate with V to find E
    reconstructed_features = np.dot(nmf_features, nmf_model.components_)
    reconstruction_error = np.abs(features - reconstructed_features)

    # Sort features by their E
    sorted_indices = np.argsort(reconstruction_error, axis=1)
    num_features_to_keep = int((100 - removal_percentage) / 100 * features.shape[1])
    selected_features = features[np.arange(features.shape[0])[:, None], sorted_indices[:, :num_features_to_keep]]
    scaler = StandardScaler()
    standardized_features = scaler.fit_transform(selected_features)

    return np.array(standardized_features)

### Loop over A and B

do a loop over steps A and B  to increase number of features

In [71]:
features_matrix = None

for i in range(10):
    features,_ = evaluation(mnist_autoencoder,mnist_test_loader)
    sorted_features = feature_filtering_nmf(features)
    
    if i == 0 :
        features_matrix = sorted_features
    else :
        features_matrix = np.concatenate((features_matrix,sorted_features),axis=1)
        
features_matrix

Evaluation loss :  0.00038030562103355623
Evaluation loss :  0.0003803095477546301
Evaluation loss :  0.0003802992422992526
Evaluation loss :  0.0003803084314148476
Evaluation loss :  0.00038031223349830213
Evaluation loss :  0.00038032574834910706
Evaluation loss :  0.0003803033115918627
Evaluation loss :  0.0003802758510418629
Evaluation loss :  0.0003802851795952128
Evaluation loss :  0.0003803151961006939


### C - Variational Autoencoder

1. # Define the dimensions

In [125]:
input_dim = 2500
hidden_dim = 500
latent_dim = 100

2. # Define the Encoders

In [134]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar

In [135]:
class SecondEncoder(nn.Module):
    def __init__(self):
        super(SecondEncoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim * 2)
        self.fc2 = nn.Linear(hidden_dim * 2, hidden_dim)
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar


In [136]:
class ThirdEncoder(nn.Module):
    def __init__(self):
        super(ThirdEncoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim // 2)
        self.fc2 = nn.Linear(hidden_dim // 2, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim * 2)
        self.fc_mu = nn.Linear(hidden_dim * 2, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim * 2, latent_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar


In [137]:
class FourthEncoder(nn.Module):
    def __init__(self):
        super(FourthEncoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim * 2)
        self.fc2 = nn.Linear(hidden_dim * 2, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.fc4 = nn.Linear(hidden_dim // 2, latent_dim)
        self.fc5 = nn.Linear(hidden_dim // 2, latent_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        mu = self.fc4(x)
        logvar = self.fc5(x)
        return mu, logvar


# Define the Decoder

# First Decoder 

In [138]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.fc2 = nn.Linear(latent_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, input_dim)
        self.sigmoid = nn.Sigmoid()  # Adding a sigmoid activation function

    def forward(self, z):
        z = F.relu(self.fc2(z))
        reconstruction = self.sigmoid(self.fc3(z))  # Applying sigmoid activation
        return reconstruction

# Second Decoder

In [ ]:
class SecondDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, input_dim):
        super(SecondDecoder, self).__init__()
        self.fc1 = nn.Linear(latent_dim, 2 * hidden_dim)
        self.fc2 = nn.Linear(2 * hidden_dim, 2 * hidden_dim)
        self.fc3 = nn.Linear(2 * hidden_dim, input_dim)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, z):
        z = self.relu(self.fc1(z))
        z = self.relu(self.fc2(z))
        reconstruction = self.sigmoid(self.fc3(z))
        return reconstruction

# Third Decoder

In [ ]:
class ThirdDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, input_dim):
        super(ThirdDecoder, self).__init__()
        self.fc1 = nn.Linear(latent_dim, 4 * hidden_dim)
        self.fc2 = nn.Linear(4 * hidden_dim, 2 * hidden_dim)
        self.fc3 = nn.Linear(2 * hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, input_dim)
        self.leaky_relu = nn.LeakyReLU()
        self.tanh = nn.Tanh()

    def forward(self, z):
        z = self.leaky_relu(self.fc1(z))
        z = self.leaky_relu(self.fc2(z))
        z = self.leaky_relu(self.fc3(z))
        reconstruction = self.tanh(self.fc4(z))
        return reconstruction

# Define the VAE

# Create instances of each encoder

In [139]:
encoder = Encoder()
second_encoder = SecondEncoder()
third_encoder = ThirdEncoder()
fourth_encoder = FourthEncoder()

# Put all encoders into a list
all_encoders = [encoder, second_encoder, third_encoder, fourth_encoder]

In [155]:
class VAE(nn.Module):
    def __init__(self, encoders):
        super(VAE, self).__init__()
        self.encoders =  nn.ModuleList(all_encoders)
        self.decoder = Decoder()

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x, encoder_idx):
        mu, logvar = self.encoders[encoder_idx](x)
        z = self.reparameterize(mu, logvar)
        reconstruction = self.decoder(z)
        return reconstruction, mu, logvar

5. # Initialize the VAE model

In [156]:
model = VAE(all_encoders)
model

VAE(
  (encoders): ModuleList(
    (0): Encoder(
      (fc1): Linear(in_features=2500, out_features=500, bias=True)
      (fc_mu): Linear(in_features=500, out_features=100, bias=True)
      (fc_logvar): Linear(in_features=500, out_features=100, bias=True)
    )
    (1): SecondEncoder(
      (fc1): Linear(in_features=2500, out_features=1000, bias=True)
      (fc2): Linear(in_features=1000, out_features=500, bias=True)
      (fc_mu): Linear(in_features=500, out_features=100, bias=True)
      (fc_logvar): Linear(in_features=500, out_features=100, bias=True)
    )
    (2): ThirdEncoder(
      (fc1): Linear(in_features=2500, out_features=250, bias=True)
      (fc2): Linear(in_features=250, out_features=500, bias=True)
      (fc3): Linear(in_features=500, out_features=1000, bias=True)
      (fc_mu): Linear(in_features=1000, out_features=100, bias=True)
      (fc_logvar): Linear(in_features=1000, out_features=100, bias=True)
    )
    (3): FourthEncoder(
      (fc1): Linear(in_features=2500, 

6. # Create a DataLoader

In [171]:
# Normalize the data to be in the [0, 1] range
max_val = features_matrix.max()
min_val = features_matrix.min()
normalized_features_matrix = (features_matrix - min_val) / (max_val - min_val)
features_tensor = torch.Tensor(normalized_features_matrix)
batch_size = 64
data_loader = DataLoader(TensorDataset(features_tensor), batch_size=batch_size, shuffle=True)

# Iterate through the data loader
for batch_idx, data in enumerate(data_loader):
    inputs = data[0]
    # Iterate through each encoder
    for encoder_idx, encoder in enumerate(all_encoders):
        reconstruction, mu, logvar = model.forward(inputs, encoder_idx)

7. # Define the loss function (using the reconstruction loss and the KL divergence)

In [172]:
def loss_function(recon_x, x, mu, logvar):
    #print(f"recon_x shape: {recon_x.shape}, x shape: {x.shape}")
    recon_loss = F.binary_cross_entropy(recon_x, x, reduction='sum')
    kld_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + kld_loss

8. # Define the optimizer

In [173]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

9. # Training loop

In [174]:
num_epochs = 10
encoder_idx = 0  # Use the first encoder

for epoch in range(num_epochs):
    total_loss = 0
    for batch_idx, data in enumerate(data_loader):
        optimizer.zero_grad()
        recon_batch, mu, logvar = model.forward(data[0], encoder_idx)  # Pass encoder_idx
        loss = loss_function(recon_batch, data[0], mu, logvar)
        loss.backward()
        total_loss += loss.item()
        optimizer.step()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(data_loader.dataset):.4f}")

Epoch 1/10, Loss: 1712.4117
Epoch 2/10, Loss: 1711.2140
Epoch 3/10, Loss: 1711.1183
Epoch 4/10, Loss: 1711.0875
Epoch 5/10, Loss: 1711.0693
Epoch 6/10, Loss: 1711.0571
Epoch 7/10, Loss: 1711.0483
Epoch 8/10, Loss: 1711.0386
Epoch 9/10, Loss: 1711.0322
Epoch 10/10, Loss: 1711.0266


In [178]:
# Set the model to evaluation mode
model.eval()

final_representations = []

# Iterate through the data loader
for batch_idx, data in enumerate(data_loader):
    inputs = data[0]  # Assuming the input data is at index 0
    with torch.no_grad():
        # Obtain the representations from the encoder part of the VAE
        mu, _ = model.encoders[encoder_idx](inputs)  # Replace encoder_idx with the desired encoder index
        final_representations.append(mu.numpy())  # Convert to numpy array and store

# Concatenate representations if required
final_representations = np.concatenate(final_representations, axis=0)
final_representations

array([[-0.00214592, -0.00082188, -0.00053966, ...,  0.00129845,
         0.00212226, -0.00081678],
       [-0.00214592, -0.00082188, -0.00053966, ...,  0.00129845,
         0.00212226, -0.00081678],
       [-0.00214592, -0.00082188, -0.00053966, ...,  0.00129845,
         0.00212226, -0.00081678],
       ...,
       [-0.00214592, -0.00082188, -0.00053966, ...,  0.00129845,
         0.00212226, -0.00081678],
       [-0.00214592, -0.00082188, -0.00053966, ...,  0.00129845,
         0.00212226, -0.00081678],
       [-0.00214592, -0.00082188, -0.00053966, ...,  0.00129845,
         0.00212226, -0.00081678]], dtype=float32)

### D- Basic Subspace Clustering

# Compute the similarity matrix using cosine similarity

In [204]:
#J'ai pas assez de ram sur mon pc mdr !!!! Ducoup je fais un PCA mais si ona le pc pour sa devrai fonctioner. PS: IL faut au loin 14GB de ram mdr donc un pc qui a 32GB de ram pour le tourner.
similarity_matrix = cosine_similarity(final_representations)
similarity_matrix


# Generate the graph Laplacian

In [191]:
degree_matrix = np.diag(np.sum(similarity_matrix, axis=1))
laplacian_matrix = degree_matrix - similarity_matrix
normalized_laplacian = np.dot(np.dot(np.sqrt(np.linalg.inv(degree_matrix)), laplacian_matrix), np.sqrt(np.linalg.inv(degree_matrix)))

TypeError: sum() received an invalid combination of arguments - got (axis=int, out=NoneType, ), but expected one of:
 * (*, torch.dtype dtype)
      didn't match because some of the keywords were incorrect: axis, out
 * (tuple of ints dim, bool keepdim, *, torch.dtype dtype)
 * (tuple of names dim, bool keepdim, *, torch.dtype dtype)


# Compute the eigenvalues and eigenvectors

In [ ]:
eigenvalues, eigenvectors = np.linalg.eigh(normalized_laplacian)
k = 10  # Example: Selecting top k eigenvectors
k_eigenvectors = eigenvectors[:, :k]

# Perform K-Means clustering on selected eigenvectors

In [ ]:
n_clusters = 5  # Example: Number of clusters
spectral_model = SpectralClustering(n_clusters=n_clusters, affinity='nearest_neighbors')
pseudo_labels = spectral_model.fit_predict(k_eigenvectors)

# Evaluate the quality of clustering

In [ ]:
silhouette_avg = silhouette_score(k_eigenvectors, pseudo_labels)